In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

In [4]:
URL = "https://medium.com/wardleymaps/finding-a-path-cdb1249078c0"

In [7]:
loader = WebBaseLoader(URL)

In [8]:
pages = loader.load_and_split()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50)
chunks = text_splitter.split_documents(pages)

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
llm = ChatOpenAI()

In [15]:
template = """SYSTEM: You are a question answer bot. 
                 Be factual in your response.
                 Respond to the following question: {question} only from 
                 the below context :{context}. 
                 If you don't know the answer, just say that you don't know.
               """
prompt = PromptTemplate.from_template(template)

In [16]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
chain.invoke("What's the size of the largest Llama 3 model?")

"I'm sorry, I don't have the specific information regarding the size of the largest Llama 3 model."

In [18]:
chain.invoke("How reading the Art of War of Sun Tzu can help in business strategy?")

'By reading the Art of War by Sun Tzu, one can gain insight into the use of topographical intelligence in business strategy. The concept of understanding the landscape, positioning, and potential obstacles can be applied to mapping out a business strategy. This can help in identifying opportunities, anticipating challenges, and making informed decisions in the competitive business environment.'